# **FASE 0: Exploración y limpieza de los datos**

**IMPORTACIÓN DE LIBRERIAS NECESARIAS PARA ESTA PRIMERA FASE**

In [1]:
import pandas as pd
import numpy as np
import datetime 
import time 

**1. CARGA DE LOS DATOS**

A continuación se cargan los seis dataframes de datos de vuelos para los proximos 3 meses, en la ruta relativa "../data", donde estan contenidas tanto las variables categoricas como numericas.

In [2]:
df_ryanair = pd.read_csv("../../data/data_ryanair.csv")
df_vbaratos = pd.read_csv("../../data/data_vuelosbaratos.csv", dtype={'PRECIO': object})
df_vbaratos1 = pd.read_csv("../../data/data_vuelosbaratosPortatil.csv", dtype={'PRECIO': object})
df_vbaratos2 = pd.read_csv("../../data/data_vuelosbaratosSobremesa.csv", dtype={'PRECIO': object})
df_vbaratos3 = pd.read_csv("../../data/data_vuelosbaratos1.csv", dtype={'PRECIO': object})
df_vbaratos4 = pd.read_csv("../../data/data_vuelosbaratos_Sobremesa.csv", dtype={'PRECIO': object})

Para poder tratar los datos en conjunto se unifican en un solo dataframe.

In [3]:
df_vbaratos['PRECIO'] = df_vbaratos['PRECIO'].apply(lambda x: x.replace('.', '').replace(',', '')).astype('float')
df_vbaratos1['PRECIO'] = df_vbaratos1['PRECIO'].apply(lambda x: x.replace('.', '').replace(',', '')).astype('float')
df_vbaratos2['PRECIO'] = df_vbaratos2['PRECIO'].apply(lambda x: x.replace('.', ',').replace(',', '')).astype('float')
df_vbaratos3['PRECIO'] = df_vbaratos3['PRECIO'].apply(lambda x: x.replace('.', '').replace(',', '')).astype('float')
df_vbaratos4['PRECIO'] = df_vbaratos4['PRECIO'].apply(lambda x: x.replace('.', '').replace(',', '')).astype('float')

In [4]:
df = pd.concat([df_ryanair, df_vbaratos, df_vbaratos1, df_vbaratos2, df_vbaratos3, df_vbaratos4 ])

In [5]:
df['PRECIO'].value_counts()

114.0     22140
125.0     20908
122.0     20840
120.0     20387
116.0     19788
          ...  
1733.0        1
2984.0        1
2985.0        1
2989.0        1
3138.0        1
Name: PRECIO, Length: 3407, dtype: int64

**2. EXPLORACIÓN INICIAL DE LOS DATOS**

**Observación de la información**

In [ ]:
#Función de la librería para que muestre todas las columnas
pd.options.display.max_columns = None

In [ ]:
df.head()

In [ ]:
#Función para enlistar todas las columnas y poder visualizarlas con mayor facilidad.
df.columns.to_list()

Tras realizarla la primera observación, se ha llevado a cabo el tratamiento de las columnas SALIDA_IDA, LLEGADA_IDA, SALIDA_VUELTA y LLEGADA_VUELTA. Para un mejor anáisis será conveniente dividir estas columnas en en fechas y horas que indiquen esta iformación por separado.

In [6]:
#Dividisión de una cadena o string a través de la función split.
df[['fecha_salida_ida','hora_salida_ida']] = df.SALIDA_IDA.str.split(expand=True)
df[['fecha_llegada_ida','hora_llegada_ida']] = df.LLEGADA_IDA.str.split(expand=True)
df[['fecha_salida_vuelta','hora_salida_vuelta']] = df.SALIDA_VUELTA.str.split(expand=True)
df[['fecha_llegada_vuelta','hora_llegada_vuelta']] = df.LLEGADA_VUELTA.str.split(expand=True)

Como enriquecimiento del dataset se decide agregar una nueva columna con el día de la semana

In [7]:
df['dia_semana'] = df['fecha_salida_ida'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d").weekday())

Para la aplicación mantenemos las columnas de fecha y hora en su forma original, como string. 

In [ ]:
# for i in df.columns[12:-1:2]:
#     df[i] = df[i].apply(lambda x: time.mktime(datetime.datetime.strptime(x, "%Y-%m-%d").timetuple()))
# df['CAPTURA'] = df['CAPTURA'].apply(lambda x: time.mktime(datetime.datetime.strptime(x, "%Y-%m-%d").timetuple()))

In [ ]:
# for i in df.columns[13:-1:2]:
#     df[i] = pd.to_datetime(df[i],format= '%H:%M' ).dt.time
#     df[i] = df[i].apply(lambda x: (x.hour*60+x.minute)*60)

**Comprobación de la existencia de valores duplicados:**


Para asegurar la fiabilidad del dataset, ya que los datos han sido extraidos por varias personas en distintos procesos, se hará un control previo en base a la redundancia de información por registros duplicados. Para ello se eliminan los valores duplicados.

In [8]:
print(df.shape)
df.drop_duplicates(keep='first', inplace=True)
print(df.shape)
df.head()

(2661808, 21)
(2484667, 21)


,Unnamed: 0,CAPTURA,FUENTE,ORIGEN,DESTINO,SALIDA_IDA,LLEGADA_IDA,SALIDA_VUELTA,LLEGADA_VUELTA,PRECIO,...,AEROLINEA,fecha_salida_ida,hora_salida_ida,fecha_llegada_ida,hora_llegada_ida,fecha_salida_vuelta,hora_salida_vuelta,fecha_llegada_vuelta,hora_llegada_vuelta,dia_semana
0,0,2020-09-09,Ryanair,BCN,AGP,2020-10-01 09:25,2020-10-01 11:20,2020-10-08 07:15,2020-10-08 08:50,29.98,...,Ryanair,2020-10-01,09:25,2020-10-01,11:20,2020-10-08,07:15,2020-10-08,08:50,3
1,1,2020-09-09,Ryanair,BCN,BGY,2020-10-01 20:15,2020-10-01 22:00,2020-10-08 22:25,2020-10-08 23:59,25.98,...,Ryanair,2020-10-01,20:15,2020-10-01,22:00,2020-10-08,22:25,2020-10-08,23:59,3
2,2,2020-09-09,Ryanair,BCN,BHX,2020-10-01 12:40,2020-10-01 14:10,2020-10-08 14:35,2020-10-08 17:45,29.61,...,Ryanair,2020-10-01,12:40,2020-10-01,14:10,2020-10-08,14:35,2020-10-08,17:45,3
3,3,2020-09-09,Ryanair,BCN,BRU,2020-10-01 06:05,2020-10-01 08:20,2020-10-08 08:55,2020-10-08 11:00,30.24,...,Ryanair,2020-10-01,06:05,2020-10-01,08:20,2020-10-08,08:55,2020-10-08,11:00,3
4,4,2020-09-09,Ryanair,BCN,BUD,2020-10-01 17:00,2020-10-01 19:40,2020-10-08 20:05,2020-10-08 22:40,31.24,...,Ryanair,2020-10-01,17:00,2020-10-01,19:40,2020-10-08,20:05,2020-10-08,22:40,3


Como se puede comprobar han sido eliminados todos los valores duplicados.

**Comprobación de la existencia de valores nulos**

Se llevará acabo una exploración de aquellos datos nulos que puedan darse en el dataset y caso afirmativo se valorará a posteriori como tratarlos.

In [ ]:
df.isnull().sum()

Se comprueba la existencia de 33 valores nulos en la columna AEROLINEA.

**3. LIMPIEZA**

Llegados a este punto se puede coprobar que es conveniente elimiar al menos aquellas columnas que ya sabemos que pueden añadir ruido al futuro análisis. En este caso serán las columnas Unnamed: 0 las columnas de fechas que han sido transformadas y la columna URL.

In [9]:
df.drop(['Unnamed: 0', 'SALIDA_IDA', 'LLEGADA_IDA', 'SALIDA_VUELTA', 'LLEGADA_VUELTA', 'URL'], axis='columns', inplace=True)

Se analizan los valores nulos y se deciden eliminar las 33 filas por ser una cifra insignificante comparada con la cantidad de información que se maneja.

In [ ]:
df[df['AEROLINEA'].isnull()]

In [10]:
#Función para eliminar filas que contendan al menos un NaN
print(df.shape)
df.dropna(axis=0, inplace = True) 
print(df.shape)

(2484667, 15)
(2484634, 15)


**4. ENRIQUECIMIENTO DEL DATASET**

**5. EXPORTACIÓN DE LOS DATOS**

Finalmente los datos serán exportados en un csv con el nombre de data_vuelos_completo.csv a la carpeta data que se encuentra en el repositorio del proyecto. Ruta relativa "../../data".

In [11]:
df.to_csv("../../data/data_vuelos_completo_app.csv", index=False)